In [35]:
import copy

In [110]:
def is_column(a):
    """
    Check if vector a is a column vector
    """
    m, n = dim(a)
    return m > 1 and n == 1

def is_row(a):
    """
    Check if vector a is a row vector
    """
    m, n = dim(a)
    return m == 1 and n > 1

def is_vector(a):
    """
    Check if a is a vector
    """
    return is_row(a) or is_column(a)

def is_scalar(a):
    """
    Check if a is a scalar
    """
    m, n = dim(a)
    return m == 1 and n == 1

In [143]:
def zeros(m,n):
    """
    Create zeros matrix of size mxn
    """
    return [[0 for i in range(n)] for j in range(m)]

def dim(A):
    """
    Return the size of matrix A
    """
    if isinstance(A, int) or isinstance(A, float):
        return 1,1
    # row vector
    elif isinstance(A, list) and not isinstance(A[0], list):
        return 1, len(A)
    elif isinstance(A[0], list) and not isinstance(A, list):
        return len(A), 1
    else:
        return len(A), len(A[0])
    
def row_mat(A,i):
    """
    Return the row at index i
    """
    return [A[i]]

def column_mat(A,j):
    """
    Return the column at index i
    """
    return [[A[i][j]] for i in range(len(A))]

def tr(A):
    """
    Calculate the trace of matrix A
    """
    if not dim(A)[0] == dim(A)[1]:
        return f"Not a square matrix {dim(A)}"
    return sum(A[i][i] for i in range(len(A)))

In [145]:
a = [[1,2,3]]
b = [[1],[2],[3]]
C = [[1,2,3],[4,5,6]]
print(transpose(a))
print(transpose(b))
print(transpose(C))
print(row_mat(C,1))
print(column_mat(C,1))
print(tr(C))

[[1], [2], [3]]
[[1, 2, 3]]
[[1, 4], [2, 5], [3, 6]]
[[4, 5, 6]]
[[2], [5]]
Not a square matrix (2, 3)


In [138]:
def transpose(A):
    """
    Transpose matrix A
    """
    n_C, m_C = dim(A)
    C = zeros(m_C, n_C)
    if is_scalar(A):
        C = [A]
    elif is_row(A):
        for i in range(len(A[0])):
            C[i][0] = A[0][i]
    elif is_column(A):
        for i in range(len(A)):
            C[0][i] = A[i][0]
    else:
        for i in range (m_C):
            for j in range (n_C):
                C[i][j] = A[j][i]
    return C

def mat_add(A,B):
    """
    Add entry-wise matrix B to A
    """
    C = copy.deepcopy(A)
    if not dim(A) == dim(B):
        return "Unequal size A and B"
    
    for i in range(len(A)):
        for j in range(len(A[0])):
            C[i][j] += B[i][j]
    return C

def mat_sub(A,B):
    """
    Subtract entry-wise matrix B from A
    """
    C = copy.deepcopy(A)
    if not dim(A) == dim(B):
        return "Unequal size A and B"
    
    for i in range(len(A)):
        for j in range(len(A[0])):
            C[i][j] -= B[i][j]
    return C

def mat_scal_mul(A, alpha):
    """
    Multiply entry-wise matrix A with scalar alpha
    """
    C = copy.deepcopy(A)
    for i in range(len(A)):
        for j in range(len(A[0])):
            C[i][j] *= alpha
    return C

def dot(a,b):
    """
    Calculate dot product between vector a and b
    """
    if not (is_vector(a) and is_vector(b)):
        return f"Invalid vector size {dim(a)} and {dim(b)}"
    if not is_row(a):
        a = transpose(a)
    if not is_column(b):
        b = transpose(b)
    if not dim(a)[1] == dim(b)[0]:
        return f"Invalid vector size {dim(a)} and {dim(b)}"
    return sum(a[0][i]*b[i][0] for i in range(len(a[0]))) 

def mat_mat_mul(A,B):
    """
    Implement matrix-matrix multiplication
    """
    if not dim(A)[1] == dim(B)[0]:
        return f"Invalid matrix size {dim(A)} and {dim(B)}"
    
    C = zeros(dim(A)[0], dim(B)[1])
    for i in range(len(A)):
        for j in range(len(B[0])):
            C[i][j] = dot(row_mat(A,i), column_mat(B,j))
    return C

def inverse(A):
    """
    Find the inverse of matrix A
    """

In [139]:
A = [
    [1,2],
    [3,4]
    ]

B = [
    [1,2],
    [3,4]
    ]

print(mat_add(A,B))
print(mat_sub(A,B))
print(mat_scal_mul(A,3))

[[2, 4], [6, 8]]
[[0, 0], [0, 0]]
[[3, 6], [9, 12]]


In [140]:
a = [[1,2,3,4]]
b = [[1],[2],[3]]
dot(a,b)

'Invalid vector size (1, 4) and (3, 1)'

In [142]:
A = [[-1,0],[2,3]]
B = [[1],[3]]
mat_mat_mul(A,B)

[[-1], [11]]